In [348]:
import json
import glob
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

## Setup parameter


In [360]:
input_file_path ='./t2dv2-dev-input'
json_save_path = './mtab_json_dev'
csv_save_path = './mtab_csv_dev'
gt_file_path = './round_1'
gt_label_path = './mtab_gt_dev'
metrics_save_path = './mtab_metrics_dev'
total_metrics = 'dev_results.csv'
files_to_join = ['./dev_predictions_metrics/metrics_1.csv','./dev_predictions_metrics/metrics_200.csv']
tags = ['tl1','tl200']
limit = 1000

In [297]:
!mkdir -p $json_save_path
!mkdir -p $csv_save_path
!mkdir -p $gt_label_path
!mkdir -p $metrics_save_path

In [259]:
json_save_path

'./mtab_json_test'

## Get MTab created json file

In [260]:
ls = glob.glob(input_file_path + "/*.csv")

for file in ls:
    json_name = json_save_path + file[len(input_file_path):len(file)-4] + '.json'
    !curl -X POST -F file=@$file https://mtab.app/api/v1/mtab?limit=$limit -o $json_name

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228k  100  212k  100 16493   1644    124  0:02:13  0:02:12  0:00:01 4875016493    0     0  100 16493      0   4586  0:00:03  0:00:03 --:--:--  4586    0    312  0:00:52  0:00:52 --:--:--     09 --:--:--     01:07  0:01:06  0:00:01     0    0    149  0:01:50  0:01:49  0:00:01     0
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24538  100 22269  100  2269   1980    201  0:00:11  0:00:11 --:--:--  4665
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30358  100 27133  100  3225   2983    354  0:00:09  0:00:09 --:--:--  7489:08 --:--:--     06  3509
  % Total    % Received % Xferd  Average Speed   Time   

## Transform json to table-linker format csv

In [303]:
ls = glob.glob( json_save_path+ "/*.json")
for file in ls:
    if os.path.getsize(file) == 0:
        continue
    f = open(file)
    data = json.load(f)
    
    # get data and input from json file to arrays 
    target = data['tables'][0]['semantic']['cea']
    inp = data['tables'][0]['table_cells']


    #load the data we want
    annotation = []
    desc= []
    kg_id = []
    kg_label = []
    wikipedia = []
    target_ls = []
    tar = []
    col = []
    for i in range(0, len(target)):
        if 'dbpedia' in target[i]['annotation']:
            annotation.append(target[i]['annotation']['dbpedia'][target[i]['annotation']['dbpedia'].find('resource/')+9:])
        else:
            annotation.append('')
        desc.append(target[i]['annotation']['desc'])
        kg_label.append(target[i]['annotation']['label'])
        kg_id.append(target[i]['annotation']['wikidata'][target[i]['annotation']['wikidata'].find('/entity/')+8:])

        if 'wikipedia' in target[i]['annotation']:
            wikipedia.append(target[i]['annotation']['wikipedia'][target[i]['annotation']['wikipedia'].find('/wiki/')+6:])
        else:
            wikipedia.append('')
        tar.append(target[i]['target'])
        target_ls.append(inp[target[i]['target'][0]][target[i]['target'][1]])

    col = []
    row= []
    for i in tar:
        row.append(i[0]-1)
        col.append(i[1])

    import pandas as pd
    df = pd.DataFrame(list(zip(col, row, target_ls, kg_id, kg_label, desc, annotation,wikipedia)),
                   columns =['column', 'row', 'label', 'kg_id', 'kg_labels','kg_descriptions', 'dbpedia', 'wikipedia'])


    df.to_csv(csv_save_path + file[len(json_save_path):len(file)-5] +'.csv', index = False)
    

## Label the results with ground truth

In [304]:
ls = glob.glob(csv_save_path+ "/*.csv")
for file in ls:
    if os.path.getsize(file) == 0:
        continue
    gt_file = gt_file_path + file[len(csv_save_path):]
    output_path = gt_label_path + file[len(csv_save_path):]
    
    !tl ground-truth-labeler -f $gt_file < $file > $output_path


ground-truth-labeler Time: 0.012091875076293945s
ground-truth-labeler Time: 0.010023117065429688s
ground-truth-labeler Time: 0.01080179214477539s
ground-truth-labeler Time: 0.011776924133300781s
ground-truth-labeler Time: 0.010269880294799805s
ground-truth-labeler Time: 0.011457204818725586s
ground-truth-labeler Time: 0.011809825897216797s
ground-truth-labeler Time: 0.01980900764465332s
ground-truth-labeler Time: 0.011887073516845703s
ground-truth-labeler Time: 0.01222085952758789s


## Get Metrics and output to file

In [324]:
ls = glob.glob( gt_label_path+ "/*.csv")
df_list = []
for file in ls:
    if os.path.getsize(file) == 0:
        continue
    output_path = metrics_save_path + file[len(gt_label_path):]
    out = file[len(gt_label_path)+1:]
    !tl metrics $file -k 1 -c "evaluation_label" --tag $out > $output_path
    if os.path.getsize(output_path) > 1:
        df = pd.read_csv(output_path)
        df_list.append(df)     

#pd.concat(df_list).to_csv(total_metrics, index = False)

metrics Time: 0.3990919589996338s
metrics Time: 0.11082911491394043s
metrics Time: 0.12528109550476074s
metrics Time: 0.3603677749633789s
metrics Time: 0.09476995468139648s
metrics Time: 0.37699031829833984s
metrics Time: 0.4293069839477539s
metrics Time: 1.0290112495422363s
metrics Time: 0.03458094596862793s
metrics Time: 0.34078001976013184s


## Join with other files

In [369]:
df = pd.concat(df_list)
df.columns =['mtab_k', 'mtab_f1','mtab_precision', 'mtab_recall', 'tag']
df = df.reset_index(drop = True)

for index in range (0, len(files_to_join)):
    file = files_to_join[index]
    print(file)
        
    df.insert(0, tags[index] + '_precision', '')
    df.insert(0, tags[index] + '_recall', '')
    df.insert(0, tags[index] + '_f1', '')
    df.insert(0, tags[index] + '_k', '')
    
    new_df = pd.read_csv(file)
    print(new_df)
    for i in range(0, len(df)):
        for j in range(0, len(new_df)):
            if df.tag[i] == new_df.tag[j]:
                print(index)
                df[tags[index] + '_k'][i]  = new_df['k'][j]
                df[tags[index] + '_f1'][i] = new_df['f1'][j]
                df[tags[index] + '_precision'][i] = new_df['precision'][j]
                df[tags[index] + '_recall'][i] = new_df['recall'][j]
                continue
                
df.to_csv(total_metrics, index = False)

./dev_predictions_metrics/metrics_1.csv
   k        f1  precision    recall                                 tag
0  1  0.948316   0.990000  0.910000  39759273_0_1427898308030295194.csv
1  1  0.981132   1.000000  0.962963  45073662_0_3179937335063201739.csv
2  1  0.572727   0.954545  0.409091  29414811_2_4773219892816395776.csv
3  1  0.795788   0.806122  0.785714  84575189_0_6365692015941409487.csv
4  1  0.974359   1.000000  0.950000  14380604_4_3329235705746762392.csv
5  1  0.889908   0.967033  0.824176  52299421_0_4473286348258170200.csv
6  1  0.952381   0.952381  0.952381   50270082_0_444360818941411589.csv
7  1  0.749972   0.754545  0.745455  28086084_0_3127660530989916727.csv
8  1  0.921063   0.981132  0.867925   14067031_0_559833072073397908.csv
0
0
0
0
0
0
0
0
0
./dev_predictions_metrics/metrics_200.csv
     k        f1  precision    recall                                 tag
0  200  0.994975   0.990000  1.000000  39759273_0_1427898308030295194.csv
1  200  1.000000   1.000000  1.0

In [363]:
df

,tl200_k,tl200_f1,tl200_recall,tl200_precision,tl1_k,tl1_f1,tl1_recall,tl1_precision,mtab_k,mtab_f1,mtab_precision,mtab_recall,tag
0,,,,,200,0.994975,1.0,0.99,1,0.990000,0.990000,0.990000,39759273_0_1427898308030295194.csv
1,,,,,200,1.0,1.0,1.0,1,0.814815,0.814815,0.814815,45073662_0_3179937335063201739.csv
2,,,,,200,0.976744,1.0,0.954545,1,0.818182,0.818182,0.818182,29414811_2_4773219892816395776.csv
3,,,,,200,0.892655,1.0,0.806122,1,0.653061,0.653061,0.653061,84575189_0_6365692015941409487.csv
4,,,,,200,1.0,1.0,1.0,1,0.900000,0.900000,0.900000,14380604_4_3329235705746762392.csv
5,,,,,200,0.967033,0.967033,0.967033,1,0.956044,0.956044,0.956044,52299421_0_4473286348258170200.csv
6,,,,,200,0.97561,1.0,0.952381,1,0.779762,0.779762,0.779762,50270082_0_444360818941411589.csv
7,,,,,200,0.844611,0.959091,0.754545,1,0.754545,0.754545,0.754545,28086084_0_3127660530989916727.csv
8,,,,,,,,,1,1.000000,1.000000,1.000000,1438042986423_95_20150728002306-00329-ip-10-23...
9,,,,,200,0.981132,0.981132,0.981132,1,0.981132,0.981132,0.981132,14067031_0_559833072073397908.csv
